##### Description:
Integrate Daily Pricing Data , Geolocation Data and Weather Data to Publish in Gold Layer as a source data for Future Price Prediction Using AI

##### Source Tables:
pricing_analytics.silver.daily_pricing_silver , pricing_analytics.silver.geo_location_silver , pricing_analytics.silver.weather_data_silver

##### Target Table name : DataLake_Price-Prediction-Gold
###### Target Table Column Mappings:
| SOURCE_TABLE_NAME | SOURCE_COLUMN_NAME | DATALAKE_TABLE_NAME | DATALAKE_COLUMN_NAME | TRANSFORMATION RULE | CONDITIONS |
| --- | --- |--- | --- |--- |--- |
| silver.daily_pricing_silver	| DATE_OF_PRICING	|datalake_price_prediction_gold| DATE_OF_PRICING| Direct Mapping |  |
| silver.daily_pricing_silver	| STATE_NAME	|datalake_price_prediction_gold| STATE_NAME| Direct Mapping |  |
| silver.daily_pricing_silver	| MARKET_NAME	|datalake_price_prediction_gold|  MARKET_NAME	| Direct Mapping |  |
| silver.daily_pricing_silver	| PRODUCTGROUP_NAME |datalake_price_prediction_gold| PRODUCT_ID| Direct Mapping |  |
| silver.daily_pricing_silver	| PRODUCT_NAME	|datalake_price_prediction_gold| PRODUCT_ID| Direct Mapping |  |
| silver.daily_pricing_silver	| VARIETY	|datalake_price_prediction_gold| VARIETY_ID|Direct Mapping |  |
| silver.daily_pricing_silver	| ROW_ID	|datalake_price_prediction_gold| ROW_ID| Direct Mapping |  |
| silver.daily_pricing_silver	| ARRIVAL_IN_TONNES	|datalake_price_prediction_gold| ARRIVAL_IN_TONNES| Direct Mapping |  |
| silver.daily_pricing_silver	| MINIMUM_PRICE	|datalake_price_prediction_gold| MINIMUM_PRICE| Direct Mapping |  |
| silver.daily_pricing_silver	| MAXIMUM_PRICE	|datalake_price_prediction_gold| MAXIMUM_PRICE| Direct Mapping |  |
| silver.daily_pricing_silver	| MODAL_PRICE	|datalake_price_prediction_gold| MODAL_PRICE| Direct Mapping |  |
| silver.geo_location_silver 	| latitude	|datalake_price_prediction_gold| MARKET_LATITUDE| Change the Source Column Name | daily_pricing_silver.STATE_NAME = geo_location_silver.stateName AND daily_pricing_silver.MARKET_NAME = geo_location_silver.marketName AND geo_location_silver.countryName = 'India' |
| silver.geo_location_silver 	| longitude	|datalake_price_prediction_gold| MARKET_LONGITUDE| Change the Source Column Name |  |
| silver.geo_location_silver 	| population	|datalake_price_prediction_gold| MARKET_POPULATION| Change the Source Column Name  |  |
| silver.weather_data_silver 	| unitOfTemparature	|datalake_price_prediction_gold| TEMPARATURE_UNIT| Change the Source Column Name  | daily_pricing_silver.MARKET_NAME = weather_data_silver.marketName AND daily_pricing_silver.DATE_OF_PRICING = weather_data_silver.weatherDate |
| silver.weather_data_silver 	| maximumTemparature	|datalake_price_prediction_gold| MARKET_MAX_TEMPARATURE | Change the Source Column Name  |  |
| silver.weather_data_silver 	| minimumTemparature	|datalake_price_prediction_gold| MARKET_MIN_TEMPARATURE | Change the Source Column Name  |  |
| silver.weather_data_silver 	| unitOfRainFall	|datalake_price_prediction_gold| RAINFALL_UNIT| Change the Source Column Name  |  |
| silver.weather_data_silver 	| rainFall	|datalake_price_prediction_gold| MARKET_DAILY_RAINFALL| Change the Source Column Name  |  |
| DERIVED	| DERIVED	|datalake_price_prediction_gold	| lakehouse_inserted_date	| Load current_timestamp() | |
| DERIVED	| DERIVED	|datalake_price_prediction_gold	| lakehouse_updated_date	| Load current_timestamp() | |


- <a href="https://docs.databricks.com/en/sql/language-manual/delta-merge-into.html" target="_blank">**MERGE TABLE** </a>

In [0]:
CREATE OR REPLACE TABLE pricing_analytics.gold.DataLake_Price_Prediction_Gold AS
WITH base_data AS (
  SELECT
    dp.DATE_OF_PRICING,
    dp.ROW_ID,
    dp.STATE_NAME,
    dp.MARKET_NAME,
    dp.PRODUCTGROUP_NAME,
    dp.PRODUCT_NAME,
    dp.VARIETY,
    dp.ORIGIN,
    dp.ARRIVAL_IN_TONNES,
    dp.MINIMUM_PRICE,
    dp.MAXIMUM_PRICE,
    dp.MODAL_PRICE,
    gl.latitude AS MARKET_LATITUDE,
    gl.longitude AS MARKET_LONGITUDE,
    gl.population AS MARKET_POPULATION,
    wd.unitOfTemparature AS TEMPARATURE_UNIT,
    wd.maximumTemparature AS MARKET_MAX_TEMPARATURE,
    wd.minimumTemparature AS MARKET_MIN_TEMPARATURE,
    wd.unitOfRainFall AS RAINFALL_UNIT,
    wd.rainFall AS MARKET_DAILY_RAINFALL
  FROM pricing_analytics.silver.daily_pricing_silver dp
  INNER JOIN pricing_analytics.silver.geo_location_silver gl
    ON dp.STATE_NAME = gl.stateName
    AND dp.MARKET_NAME = gl.marketName
    AND gl.countryName = 'India'
  INNER JOIN pricing_analytics.silver.weather_data_silver wd
    ON wd.marketName = dp.MARKET_NAME
    AND dp.DATE_OF_PRICING = wd.weatherDate
)
SELECT *,
       current_timestamp() AS lakehouse_inserted_date,
       current_timestamp() AS lakehouse_updated_date
FROM base_data;


In [0]:
%sql
select count(*) from pricing_analytics.gold.DataLake_Price_Prediction_Gold

In [0]:
SELECT * FROM pricing_analytics.gold.DataLake_Price_Prediction_Gold
--WHERE MARKET_POPULATION IS NOT NULL